In [19]:
import pandas as pd
data_root = './gtfs_b/'
import os

stops_dirc = os.path.join(data_root + 'stops.txt')
routes_dirc = os.path.join(data_root + 'routes.txt')
trips_dirc = os.path.join(data_root + 'trips.txt')
stop_times_dirc = os.path.join(data_root + 'stop_times.txt')
shapes_dirc = os.path.join(data_root + 'shapes.txt')

stops = pd.read_csv(stops_dirc)
routes = pd.read_csv(routes_dirc)
trips = pd.read_csv(trips_dirc)
stop_times = pd.read_csv(stop_times_dirc)
shapes = pd.read_csv(shapes_dirc)


print("Stops:")
print(stops.head())
print("\nRoutes:")
print(routes.head())
print("\nTrips:")
print(trips.head())
print("\nStop Times:")
print(stop_times.head())
print("\nShapes:")
print(shapes.head())


Stops:
   stop_id                   stop_name  stop_desc   stop_lat   stop_lon  \
0   300000  ORIENTAL BLVD/MACKENZIE ST        NaN  40.578350 -73.940029   
1   300002    ORIENTAL BLVD/JAFFRAY ST        NaN  40.578066 -73.943029   
2   300003   ORIENTAL BLVD/HASTINGS ST        NaN  40.577909 -73.944643   
3   300004   ORIENTAL BLVD/FALMOUTH ST        NaN  40.577718 -73.946200   
4   300006      ORIENTAL BLVD/DOVER ST        NaN  40.577353 -73.949552   

   zone_id  stop_url  location_type  parent_station  
0      NaN       NaN              0             NaN  
1      NaN       NaN              0             NaN  
2      NaN       NaN              0             NaN  
3      NaN       NaN              0             NaN  
4      NaN       NaN              0             NaN  

Routes:
  route_id agency_id route_short_name                   route_long_name  \
0       B1  MTA NYCT               B1       Bay Ridge - Manhattan Beach   
1      B11  MTA NYCT              B11             Sunset Pa

In [26]:

express_routes = routes[routes['route_type'] == 3]  
print("Express Routes:")
print(express_routes)
print("routes type", routes['route_type'].unique())

Express Routes:
    route_id agency_id route_short_name  \
0         B1  MTA NYCT               B1   
1        B11  MTA NYCT              B11   
2        B12  MTA NYCT              B12   
3        B13  MTA NYCT              B13   
4        B14  MTA NYCT              B14   
..       ...       ...              ...   
269      X38  MTA NYCT              X38   
270      X63  MTA NYCT              X63   
271      X64  MTA NYCT              X64   
272      X68  MTA NYCT              X68   
291    JBLAF  MTA NYCT            JBLAF   

                                       route_long_name  \
0                          Bay Ridge - Manhattan Beach   
1                                Sunset Park - Midwood   
2                     Lefferts Gardens - East New York   
3                      Spring Creek - Wyckoff Hospital   
4                         Spring Creek - Crown Heights   
..                                                 ...   
269  Sea Gate / Bensonhurst - Midtown Manhattan Exp...   
270

In [27]:

express_trips = trips[trips['route_id'].isin(express_routes['route_id'])]
print("Express Trips:")
print(express_trips)


express_stop_times = stop_times[stop_times['trip_id'].isin(express_trips['trip_id'])]
express_stops = stops[stops['stop_id'].isin(express_stop_times['stop_id'])]
print("Express Stops:")
print(express_stops)


express_shapes = shapes[shapes['shape_id'].isin(express_trips['shape_id'])]
print("Express Shapes:")
print(express_shapes)


Express Trips:
      route_id          service_id                           trip_id  \
0         B82+       EN_O4-Weekday    EN_O4-Weekday-028500_SBS82_901   
1         B82+       EN_O4-Weekday    EN_O4-Weekday-034800_SBS82_901   
2         B82+       EN_O4-Weekday    EN_O4-Weekday-039800_SBS82_913   
3         B82+       EN_O4-Weekday    EN_O4-Weekday-047300_SBS82_913   
4         B82+       EN_O4-Weekday    EN_O4-Weekday-029700_SBS82_902   
...        ...                 ...                               ...   
57695       B6  UP_D4-Weekday-SDon  UP_D4-Weekday-SDon-136000_B6_300   
57696       B6  UP_D4-Weekday-SDon  UP_D4-Weekday-SDon-137200_B6_309   
57697       B6  UP_D4-Weekday-SDon  UP_D4-Weekday-SDon-093500_B6_232   
57698       B6  UP_D4-Weekday-SDon  UP_D4-Weekday-SDon-093700_B6_305   
57699       B6  UP_D4-Weekday-SDon  UP_D4-Weekday-SDon-094000_B6_235   

                                  trip_headsign  direction_id  block_id  \
0      SELECT BUS BENSNHRST BAY 38 via FLATLN

In [22]:
save_dir = './gtfs_b_d3data/express/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
express_routes.to_csv(save_dir + 'filtered_routes.csv', index=False)
express_trips.to_csv(save_dir + 'filtered_trips.csv', index=False)
express_stop_times.to_csv(save_dir + 'filtered_stop_times.csv', index=False)
express_stops.to_csv(save_dir + 'filtered_stops.csv', index=False)
express_shapes.to_csv(save_dir + 'filtered_shapes.csv', index=False)

print("Data processing complete. Files saved.")



Data processing complete. Files saved.


In [ ]:
#make sure each route_id and direction_id corresponds to a unique shape_id
unique_express_trips = express_trips.drop_duplicates(subset=['route_id', 'direction_id'])

In [ ]:
#load bus capacity data
capacity_data_dir = './MTA_Express_Bus_Capacity__April_2023_-_September_2023_20241127.csv' 
capacity = pd.read_csv(capacity_data_dir)

#consistent direction values
direction_mapping = {
    "NB": 0,
    "SB": 1,
    "EB": 0,
    "WB": 1
}
capacity['direction_id'] = capacity['Direction'].map(direction_mapping)
capacity.rename(columns={'Route': 'route_id'}, inplace=True)
capacity.rename(columns={'Day Type': 'Day_Type'}, inplace=True)
capacity.rename(columns={'Load Percentage': 'Load_percentage'}, inplace=True)
#filter by route_id and direction
#merge with route color and text color
filtered_capacity_data = capacity.merge(
    express_routes[['route_id', 'route_color','route_text_color']],  # choose route_id  route_color 
    left_on='route_id',                          # capacity_data--route 
    right_on='route_id',                      # route_data--route_id 
    how='inner'                               # inner join only keep matched rows
)
merge_data = filtered_capacity_data.merge(
    unique_express_trips[['route_id', 'direction_id', 'shape_id']],  # express_trips 
    left_on=['route_id', 'direction_id'],  #  filtered_capacity_data 
    right_on=['route_id', 'direction_id'],  # express_trips
    how='inner'  
)


# filtered_capacity_data = filtered_capacity_data.drop_duplicates().reset_index(drop=True)

# filtered_capacity_data.to_csv('filtered_capacity_data.csv', index=False)  
merge_data.to_csv('merge_data.csv', index=False)  


In [ ]:
#group by shape_id and merge with shape data
shape_capacity_data = pd.merge(
    express_shapes,
    merged_data,
    on='shape_id',
    how='inner'
)
shape_capacity_data = shape_capacity_data[['shape_id', 'Hour', 'shape_pt_lat', 'shape_pt_lon', 'Load Percentage']]
def map_to_time_period(hour):
    if 5 <= hour <= 12:
        return "Morning"
    elif 13 <= hour <= 20:
        return "Afternoon"
    else:
        return "Evening"

shape_capacity_data['Time Period'] = shape_capacity_data['Hour'].apply(map_to_time_period)

grouped_by_period = {}
for shape_id, group in shape_capacity_data.groupby('shape_id'):
    grouped_by_period[shape_id] = group.groupby('Time Period').apply(lambda g: g.to_dict('records')).to_dict()




In [ ]:
import pandas as pd
data_root = './gtfs_b/'
import os


routes_dirc = os.path.join(data_root + 'routes.txt')
trips_dirc = os.path.join(data_root + 'trips.txt')
shapes_dirc = os.path.join(data_root + 'shapes.txt')

routes = pd.read_csv(routes_dirc)
trips = pd.read_csv(trips_dirc)
shapes = pd.read_csv(shapes_dirc)


express_routes = routes[routes['route_type'] == 3]  

express_trips = trips[trips['route_id'].isin(express_routes['route_id'])]
print("Express Trips:")
print(express_trips)


express_shapes = shapes[shapes['shape_id'].isin(express_trips['shape_id'])]
print("Express Shapes:")
print(express_shapes)

save_dir = './gtfs_b_d3data/express/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
express_routes.to_csv(save_dir + 'filtered_routes.csv', index=False)
express_trips.to_csv(save_dir + 'filtered_trips.csv', index=False)
express_shapes.to_csv(save_dir + 'filtered_shapes.csv', index=False)

print("Data processing complete. Files saved.")


unique_express_trips = express_trips.drop_duplicates(subset=['route_id', 'direction_id'])


#load bus capacity data
capacity_data_dir = './MTA_Express_Bus_Capacity__April_2023_-_September_2023_20241127.csv'  
capacity = pd.read_csv(capacity_data_dir)

#consistent direction values
direction_mapping = {
    "NB": 0,
    "SB": 1,
    "EB": 0,
    "WB": 1
}
capacity['direction_id'] = capacity['Direction'].map(direction_mapping)
capacity.rename(columns={'Route': 'route_id'}, inplace=True)
capacity.rename(columns={'Day Type': 'Day_Type'}, inplace=True)
capacity.rename(columns={'Load Percentage': 'Load_percentage'}, inplace=True)
#filter by route_id and direction
#merge with route color and text color
filtered_capacity_data = capacity.merge(
    express_routes[['route_id', 'route_color','route_text_color']],  # choose route_id  route_color 
    left_on='route_id',                          # capacity_data--route 
    right_on='route_id',                      # route_data--route_id 
    how='inner'                               # inner join only keep matched rows
)
merge_data = filtered_capacity_data.merge(
    unique_express_trips[['route_id', 'direction_id', 'shape_id']],  # express_trips 
    left_on=['route_id', 'direction_id'],  #  filtered_capacity_data 
    right_on=['route_id', 'direction_id'],  # express_trips
    how='inner'  
)


merge_data.to_csv('merge_data.csv', index=False)  

#group by shape_id and merge with shape data
shape_capacity_data = pd.merge(
    express_shapes,
    merged_data,
    on='shape_id',
    how='inner'
)
shape_capacity_data = shape_capacity_data[['shape_id', 'Hour', 'shape_pt_lat', 'shape_pt_lon', 'Load Percentage']]
def map_to_time_period(hour):
    if 5 <= hour <= 12:
        return "Morning"
    elif 13 <= hour <= 20:
        return "Afternoon"
    else:
        return "Evening"

shape_capacity_data['Time Period'] = shape_capacity_data['Hour'].apply(map_to_time_period)

grouped_by_period = {}
for shape_id, group in shape_capacity_data.groupby('shape_id'):
    grouped_by_period[shape_id] = group.groupby('Time Period').apply(lambda g: g.to_dict('records')).to_dict()